In [2]:
import pandas as pd

results = pd.read_csv(f"../../../data/results.csv")
drivers = pd.read_csv(f"../../../data/drivers.csv")
races = pd.read_csv(f"../../../data/races.csv")

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
10249,10307,1132,822,15,77,16,1:32.431,\N,\N
10250,10308,1132,825,210,20,17,1:32.905,\N,\N
10251,10309,1132,839,214,31,18,1:34.557,\N,\N
10252,10310,1132,815,9,11,19,1:38.348,\N,\N


In [ ]:
df = pd.DataFrame()

df["driver_name"] = drivers["forename"] + " " + drivers["surname"]
df["driverId"] = drivers["driverId"]
df = df.merge(results, on="driverId")
df = df.merge(races, on="raceId")
df["race_name"] = df["name"]

# ajout des performances des courses précedentes


df["perf_n1"] = df.groupby("driverId")["positionOrder"].shift(1)
df["perf_n2"] = df.groupby("driverId")["positionOrder"].shift(2)
df["perf_n3"] = df.groupby("driverId")["positionOrder"].shift(3)

# Vitesse moyenne sur circuit


# Calcul de la vitesse moyenne cumulative par race_name, driver_name et year
df.sort_values(["driver_name", "race_name", "year"], inplace=True)

# Initialisation d'une colonne pour stocker la vitesse moyenne cumulative
df["avg_speed_cumulative"] = None

# Boucle sur chaque combinaison de driver_name et race_name
for (driver, race), group in df.groupby(["driver_name", "race_name"]):
    cumulative_speeds = []  # Liste pour stocker les vitesses cumulées
    for i, row in group.iterrows():
        # Filtrer les données jusqu'à l'année en cours (exclue)
        past_data = group[group["year"] < row["year"]]
        if not past_data.empty:
            # Calcul de la vitesse moyenne cumulative
            avg_speed = past_data["milliseconds"].mean()
        else:
            avg_speed = None  # Pas de données pour les années précédentes
        cumulative_speeds.append(avg_speed)

    # Ajouter les vitesses cumulées au DataFrame
    df.loc[group.index, "avg_speed_cumulative"] = cumulative_speeds

df

TypeError: Could not convert string '\N' to numeric